# Connecting to a Database Using Environment Variables

Hardcoding credentials (usernames, passwords, API keys) directly in your code is a **security risk**:
- If you push your notebook to GitHub, anyone can see your password
- Different environments (dev, production) need different credentials
- Rotating a password means editing every file that uses it

The solution: store credentials in a **`.env` file** and load them at runtime with `python-dotenv`.

## How It Works

| Component | Purpose |
|---|---|
| `.env` file | Stores credentials as `KEY=VALUE` pairs (one per line) |
| `.gitignore` | Lists `.env` so it is **never** committed to version control |
| `python-dotenv` | Reads the `.env` file and loads values into `os.environ` |
| `os.getenv()` | Retrieves a value by its key name |

## Step 1 — The `.env` File

Create a file named `.env` in the **project root** (same folder as your notebooks). It should look like this:

```
DB_USERNAME=your_username
DB_PASSWORD=your_password
DB_HOST=52.211.123.34
DB_PORT=25010
DB_NAME=IEMASTER
```

> **Important:** The `.env` file is already listed in `.gitignore`, so it will not be pushed to GitHub. Each student creates their own `.env` locally.

## Step 2 — Load the `.env` File

In [1]:
import os
from dotenv import load_dotenv

# load_dotenv() searches for a .env file starting from the current directory
# and loads its contents into os.environ
load_dotenv()

True

## Step 3 — Read Individual Variables

Once loaded, each key from the `.env` file is available via `os.getenv()`.

In [2]:
db_username = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# Verify the variables loaded (never print passwords in production!)
print(f"Connecting as '{db_username}' to {db_host}:{db_port}/{db_name}")

Connecting as 'daniel' to 52.211.123.34:25010/IEMASTER


## Step 4 — Build the Connection and Query

We use an **f-string** to assemble the SQLAlchemy connection URL from the variables.

In [3]:
import pandas as pd
from urllib.parse import quote_plus
from sqlalchemy import create_engine

# quote_plus() encodes special characters in the password (e.g. "@" becomes "%40")
# so they don't break the connection URL
engine = create_engine(
    f"db2+ibm_db://{db_username}:{quote_plus(db_password)}@{db_host}:{db_port}/{db_name}"
)

# Quick test: list the tables in the IEPLANE schema
query = """
SELECT TABNAME, CARD AS ROW_COUNT
FROM SYSCAT.TABLES
WHERE TABSCHEMA = 'IEPLANE'
ORDER BY TABNAME
"""

pd.read_sql(query, engine)

,tabname,row_count
0,AIRPLANES,120
1,AIRPORTS,30
2,COUNTRIES,196
3,DEPARTMENT,22
4,EMPLOYEE,1598
5,FLIGHTS,297471
6,PASSENGERS,50000
7,ROUTES,59
8,TICKETS,35383337


## Before vs. After

| | Hardcoded | With dotenv |
|---|---|---|
| **Code** | `create_engine("db2+ibm_db://daniel:db2dani@...")` | `create_engine(f"db2+ibm_db://{db_username}:{db_password}@...")` |
| **Security** | Password visible in notebook | Password stays in `.env` (not committed) |
| **Collaboration** | Everyone sees your credentials | Each person uses their own `.env` |
| **Maintenance** | Change password → edit every file | Change password → edit one `.env` file |

---

### Checklist for Your Setup

1. Create a `.env` file in the project root with your credentials
2. Confirm `.env` appears in `.gitignore`
3. Use `load_dotenv()` + `os.getenv()` in every notebook that needs database access